In [1]:
def get_soundcloud_link():
    while True:
        try: # Loop até o input ser válido
            usuario_link = input("Insira o link do perfil do SoundCloud: ").strip()
            
            # Remove "http://" ou "https://" do início, se existir
            if usuario_link.startswith("https://"):
                usuario_link = usuario_link[len("https://"):]
            elif usuario_link.startswith("http://"):
                usuario_link = usuario_link[len("http://"):]

            # Verifica se o link é do SoundCloud
            if not usuario_link.startswith("soundcloud.com"):
                raise ValueError("O link inserido não parece ser do SoundCloud.")
            
            # Remove parâmetros de URL, se existirem
            base_url = usuario_link.split("?")[0]
            path_parts = base_url.split("/")  # Divide o link por "/"

            # Mantém apenas a parte até o nome do artista (segunda posição)
            if len(path_parts) < 2 or not path_parts[1]:  # Verifica se o formato é correto
                raise ValueError("O link deve conter o nome do artista.")
            
            # Retorna o link formatado no estilo https://soundcloud.com/artista
            return f"{path_parts[0]}/{path_parts[1]}"
    
        except ValueError as ve:  # Captura a exceção e imprime a mensagem correta
            print(f"Erro: {ve}. Por favor, insira um link válido.")

get_soundcloud_link()

'soundcloud.com/pzzs'